<a href="https://colab.research.google.com/github/jnetoass/autoencoder/blob/main/denoising_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.graph_objects as go #criação de gráficos
from plotly.subplots import make_subplots  #criação de múltiplos grágicos
import plotly.express as px # criação de gráficos
from skimage import io # leitura de imagens
from random import randint # gera números aleatórios

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [ ]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor + np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor + np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [ ]:
def visualiza_numero():
  num = randint(0,60000)
  fig = px.imshow(x_train_noisy[num],color_continuous_scale='gray')
  fig.update_layout(title={
                        'text': f"o número escrito é {y_train[num]}",
                        'y':0.95,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'
                        })
  fig.show() 

In [ ]:
visualiza_numero()

In [ ]:
# Can our autoencoder learn to recover the original digits. We will use a slightly different model with more filters per layer
input_img = tf.keras.layers.Input(shape=(28, 28, 1))

x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)

# the representation is (7, 7, 32)

x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = tf.keras.models.Model(input_img, decoded)
autoencoder.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
                metrics=['accuracy'])

In [ ]:
autoencoder.fit(x_train_noisy, x_train, epochs=100, batch_size=32, shuffle=True, validation_data=(x_test_noisy, x_test))

Epoch 1/100
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1904 - accuracy: 0.7993 - val_loss: 0.1732 - val_accuracy: 0.8011
Epoch 2/100
1875/1875 [==============================] - 16s 9ms/step - loss: 0.1725 - accuracy: 0.8017 - val_loss: 0.1683 - val_accuracy: 0.8005
Epoch 3/100
1875/1875 [==============================] - 16s 9ms/step - loss: 0.1695 - accuracy: 0.8022 - val_loss: 0.1661 - val_accuracy: 0.8024
Epoch 4/100
1875/1875 [==============================] - 16s 9ms/step - loss: 0.1680 - accuracy: 0.8025 - val_loss: 0.1656 - val_accuracy: 0.8009
Epoch 5/100
1875/1875 [==============================] - 16s 9ms/step - loss: 0.1669 - accuracy: 0.8027 - val_loss: 0.1646 - val_accuracy: 0.8028
Epoch 6/100
1875/1875 [==============================] - 16s 9ms/step - loss: 0.1663 - accuracy: 0.8028 - val_loss: 0.1649 - val_accuracy: 0.8011
Epoch 7/100
1875/1875 [==============================] - 16s 9ms/step - loss: 0.1658 - accuracy: 0.8028 - val_loss: 0.1638 -

In [ ]:
decoded_imgs = autoencoder.predict(x_test)

In [ ]:
x_test.shape, decoded_imgs.shape

((10000, 28, 28), (10000, 28, 28, 1))

In [ ]:
decoded_imgs = decoded_imgs[:,:,:,0]

In [ ]:
random_number = randint(0,10000)
#img1 = io.imread(images[random_number])
#img2 = io.imread(augmented_images[random_number])

fig = make_subplots(
    rows=1, cols=3)
fig_img1 = px.imshow(x_test_noisy[random_number], color_continuous_scale='gray')
fig_img2 = px.imshow(decoded_imgs[random_number], color_continuous_scale='gray')
fig_img3 = px.imshow(x_test[random_number], color_continuous_scale='gray')
fig.add_trace(fig_img1.data[0], 1, 1)
fig.add_trace(fig_img2.data[0], 1, 2)
fig.add_trace(fig_img3.data[0], 1, 3)
fig.show()